(c) Juan Gomez 2019. Thanks to Universidad EAFIT for support. This material is part of the course Introduction to Finite Element Analysis

# Global assembly of the elemental FEM equilibrium equations

## Introduction

In this Notebook we describe computational details related to the final assembly of the global equilibrium equations in the finite element formulation of the theory of elasticity boundary value problem. This assembly process corresponds to the addition of the stiffness matrices corresponding to each element of the mesh considering the appropriate position of each coeeficient. **After completing this notebook you should be able to:**

* Understand the physical basis, in terms of the action-reaction principle, behind the process of assembly of global equilibrium equations.

* Recognize the main numerical operators involved in the process of assembly of global equilibrium equations.

* Implement algorithms to assemble global equilibrium equations for general elements.

## Finite element assembly

Consider the simple finite element model shown in the figure below. As discussed previously, and as a result of discretizing the PVW, the nodal forces associated to the $Q$-th degree of freedom satisfy the following equilibrium relationship:

$$
K^{QP}u^P=f_V^Q+f_t^Q.
$$


The term $K^{QP}u^P$ in this equilibrium equation corresponds to the nodal force $f_\sigma^Q$ resulting from internal forces associated to the element stresses. The total magnitude of these internal forces its due to the contribution from all the elements connecting to the node. This is exactly the same case when solving a simple spring-mass system, [see for instance Bathe(2006) Example 3.1].

<center><img src="img/assembled.png" alt="files" style="width:500px"></center>



The process of considering internal force terms like $f_\sigma^Q$, accounting for all the elements, and leading to the global equilibrium equations of the system is called the assembly process. The resulting internal forces for the complete system $\left\{F_\sigma\right\}$ can be written in organized form like:


$$
\left\{F_\sigma\right\}=\left[K^G\right]\left\{U^G\right\}
$$

and the equilibriun equations for the finite element model as:

$$
\left\{F_\sigma\right\}-\left\{F_V\right\}-\left\{F_t\right\}=0
$$

where $\left\{F_V\right\}$ and $\left\{F_t\right\}$ are global force vectors due to body forces and surface tractions. The assembly of the global stiffness matrix $\left[K^G\right]$ leading to the internal forces vector can be written like:

$$
\left[K^G\right]=\bigwedge_{i=1}^{Numel} k^i
$$

where $\bigwedge$ is called the **assembly operator** which loops through the $NUMEL$ elements in the mesh and adds each local coefficient matrix $k^i$. The assembly operator works like the standard summation operator $\Sigma$ but with the intrinsic inteligence of adding the terms at the right positions.

## Physical assembly

The process of assembly can be easily understood considering Newton's third law of action and reaction. This is ilustrated in the figure below where we have labeled $U_b$ those degrees of freedom along the common surface $S_b$ and $U_a$ and $U_c$ those pertaining to other regions of element $1$ and $2$ respectively.

<center><img src="img/coupled1.png" alt="files" style="width:500px"></center>


Now, the nodal forces representing the internal stresses take the following forms in each element:


$$
\begin{Bmatrix}F_a\\F_b\end{Bmatrix} = \begin{bmatrix}K_{aa}^1&K_{ab}^1\\K_{ba}^1&K_{bb}^1\end{bmatrix}\begin{Bmatrix}U_a\\U_b\end{Bmatrix}
$$

and

$$
\begin{Bmatrix}-F_b\\F_c\end{Bmatrix}=\begin{bmatrix}K_{bb}^2&K_{bc}^2\\K_{cb}^2&K_{cc}^2\end{bmatrix}\begin{Bmatrix}U_b\\U_c\end{Bmatrix}.
$$


Using the equilibrium and compatibility conditions in terms of nodal forces and displacements given by:

\begin{align*}
& F_b^1+F_b^2=0
& U_b^1=U_b^2
\end{align*}

yields the equilibrium equations for the two element assemblage:

$$
\begin{bmatrix}K_{aa}^1&K_{ab}^1&0\\K_{ba}^1&K_{bb}^1+K_{bb}^2&K_{bc}^2\\0&K_{cb}^2&K_{cc}^2\end{bmatrix}\begin{Bmatrix}U_a\\U_b\\U_c\end{Bmatrix}=\begin{Bmatrix}F_a\\0\\F_c\end{Bmatrix}.
$$


The addition of more elements via mechanical interaction through the exposed surfaces implies the same process of canceling force terms and enforcing displacement compatibility. At the end of the process the only forces left are those introduced by surface tractions and body forces.

**Questions:**

***For the mesh shown in the figure, with internal surfaces between elements 1-3 and 3-2 labeled $S_b$ and $S_c$ respectively, write the form of the global stiffness matrix resulting from the physical assembly. Explicitly formulate the force and displacement compatibility equations along both boundaries.**



<center><img src="img/long.png" alt="files" style="width:300px"></center>


## Computational assembly

Computationally, the assembly process implies (i) identifying active and restrained degrees of freedom (dof) in the mesh (ii) assigning equation identifiers to the active degrees of freedom and (iii) identifying the contributtion from each element to the different degrees of freedom.

### Boundary conditions array IBC()

To identify active and restrained dofs the nodal data specifies a bounadry condition index to each node (see figure) with values $0$ and $-1$ specifying a free and restrained dof respectively. So the nodal data in the input file gives for each node its nodal id, the nodal coordinates in the global reference system and the boundary condition flag.

<center><img src="img/nodesF.png" alt="files" style="width:200px"></center>


The boundary conditions data is then stored into an integer array **IBC()** which in a first instance contains only $0$s and $-1$s

$$
\begin{array}{c}0\\1\\2\\3\\4\\5\\6\\7\\8\end{array}\begin{bmatrix}0&-1\\-1&-1\\0&-1\\0&0\\0&0\\0&0\\0&0\\0&0\\0&0\end{bmatrix}
$$

and in a second instance is transformed into equation numbers:


$$
\begin{array}{c}0\\1\\2\\3\\4\\5\\6\\7\\8\end{array}\begin{bmatrix}0&-1\\-1&-1\\1&-1\\2&3\\4&5\\6&7\\8&9\\10&11\\12&13\end{bmatrix}
$$




The following two subroutines read the input (text) files (nodes, mats , elements and loads) and form the boundary conditions array **IBC()** in its two instances completing steps (i) and (ii) for the computational assembly process. This last step is performed by the subroutine **eqcounter()**.

In [1]:
%matplotlib inline        
import matplotlib.pyplot as plt
import numpy as np
import sympy as sym

**(Add comments to clarify the relevant steps ion the code below)**.

In [2]:
def readin():
    nodes    = np.loadtxt('files/' + 'snodes.txt', ndmin=2)
    mats     = np.loadtxt('files/' + 'smater.txt', ndmin=2)
    elements = np.loadtxt('files/' + 'seles.txt', ndmin=2, dtype=np.int)
    loads    = np.loadtxt('files/' + 'sloads.txt', ndmin=2)

    return nodes, mats, elements, loads
nodes, mats, elements, loads = readin()

In [3]:
def eqcounter(nodes):

    nnodes = nodes.shape[0]
    IBC = np.zeros([nnodes, 2], dtype=np.integer)
    for i in range(nnodes):
        for k in range(2):
            IBC[i , k] = int(nodes[i , k+3])
    neq = 0
    for i in range(nnodes):
        for j in range(2):
            if IBC[i, j] == 0:
                IBC[i, j] = neq
                neq = neq + 1

    return neq, IBC
neq, IBC = eqcounter(nodes)

### Element connectivites array IELCON()

Step (iii) in the process is completed after relating nodes in each element to the equation numbers specified in **IBC()**. The nodal points defining each element are input in a data file (see figure below). Note that each nodal identifier indicates the row in the **IBC()** array storing the equation numbers assigned to this node.


<center><img src="img/elesF.png" alt="files" style="width:400px"></center>


The nodal data for each element is stored in a connectivities array **IELCON()** where the row and element number coincide.


$$
\begin{array} {c}0\\1\\2\\3\end{array}\begin{bmatrix}0&1&4&3\\3&4&7&6\\4&5&8&7\\1&2&5&4\end{bmatrix}
$$

**Question:**

**Modify the node ordering in the definition of each elementand explain what would be the implications of this change in the local stiffness matrix.**

### The assembly operator DME() array

The final step in the construction of the assembly operator, termed here the **DME()** operator is just the translation of the **IELCON()** array storing nodal numbers into equation numbers stored in **IBC()**:

$$
\begin{array}{c}0\\1\\2\\3\end{array}\begin{bmatrix}0&-1&-1&-1&4&5&2&3\\2&3&4&5&10&11&8&9\\4&5&6&7&12&13&10&11\\-1&-1&1&-1&6&7&4&5\end{bmatrix}
$$


**Question:**

**(i) Use the IELCON() array together with the boundary conditions array IBC() to find the assembly operator.**

**(ii) Use a different numberig scheme for the sample mesh shown above and repeat the computation of the assebly operator.**

The **DME()** operator can now be used in a straight forward process relating local to global equations identifiers. For instance the first row of the stiffness matrix for element 2 is assembled as indicated next:


$$
\begin{align*}
K_{22}^G & \leftarrow K_{22}^G+k_{00}^2\\
K_{23}^G & \leftarrow K_{23}^G+k_{01}^2\\
K_{24}^G & \leftarrow K_{24}^G+k_{02}^2\\
K_{25}^G & \leftarrow K_{25}^G+k_{03}^2\\
K_{2,10}^G & \leftarrow K_{2,10}^G+k_{04}^2\\
K_{2,11}^G & \leftarrow K_{2,11}^G+k_{05}^2\\
K_{28}^G & \leftarrow K_{28}^G+k_{06}^2\\
K_{29}^G & \leftarrow K_{29}^G+k_{07}^2
\end{align*}
$$

The **DME()** operator is obtained by the following subroutine which takes as input arguments the nodes and elements arrays and returns the assembly operator.

**(Add comments to clarify the relevant steps in the code below)**.

In [4]:
def DME(nodes, elements):

    nels = elements.shape[0]
    IELCON = np.zeros([nels, 4], dtype=np.integer)
    DME = np.zeros([nels, 8], dtype=np.integer)

    neq, IBC = eqcounter(nodes)
    ndof   = 8
    nnodes = 4
    ngpts  = 4
    for i in range(nels):
        for j in range(nnodes):
            IELCON[i, j] = elements[i, j+3]
            kk = IELCON[i, j]
            for l in range(2):
                DME[i, 2*j+l] = IBC[kk, l]

    return DME , IBC , neq

In [5]:
DME , IBC , neq = DME(nodes, elements)
print(DME)

[[ 0 -1 -1 -1  4  5  2  3]
 [ 2  3  4  5 10 11  8  9]
 [ 4  5  6  7 12 13 10 11]
 [-1 -1  1 -1  6  7  4  5]]


It was shown that the assembly involves a typical step like:

$$
K_{22}^G \leftarrow K_{22}^G+k_{00}^2
$$

which involves computation of local elemental matrices with terms $K_{ij}^q$. The following subroutine uses as input the **DME()** operator and loops through the elements of the mesh to compute the local matrix [see **UEL()**] and add its contribution into the global matrix.

**(Add comments to clarify the relevant steps ion the code below)**.

In [6]:
def assembly(elements, mats, nodes, neq, DME, uel=None):

    IELCON = np.zeros([4], dtype=np.integer)
    KG = np.zeros((neq, neq))
    nels = elements.shape[0]
    nnodes = 4
    ndof = 8
    for el in range(nels):
        elcoor = np.zeros([nnodes, 2])
        im     = np.int(elements[el , 2])
        par0, par1 = mats[im , :]
        for j in range(nnodes):
            IELCON[j] = elements[el , j+3]
            elcoor[j, 0] = nodes[IELCON[j], 1]
            elcoor[j, 1] = nodes[IELCON[j], 2]
        kloc = uel4nquad(elcoor, par1, par0)
        dme = DME[el, :ndof]
        for row in range(ndof):
            glob_row = dme[row]
            if glob_row != -1:
                for col in range(ndof):
                    glob_col = dme[col]
                    if glob_col != -1:
                        KG[glob_row, glob_col] = KG[glob_row, glob_col] +\
                                                 kloc[row, col]

    return KG

In this case we have assumed that the elemental subroutine produces a stiffness matrix filled with $1$s.

**(Complete this suboroutine with the implementation performed in NB 8)**.

In [7]:
def uel4nquad(coord, enu, Emod):

    kl = np.ones([8, 8])
    return kl

In [8]:
KG = assembly(elements, mats, nodes, neq, DME)
print(KG)

[[1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 2. 2. 2. 2. 0. 0. 1. 1. 1. 1. 0. 0.]
 [1. 0. 2. 2. 2. 2. 0. 0. 1. 1. 1. 1. 0. 0.]
 [1. 1. 2. 2. 4. 4. 2. 2. 1. 1. 2. 2. 1. 1.]
 [1. 1. 2. 2. 4. 4. 2. 2. 1. 1. 2. 2. 1. 1.]
 [0. 1. 0. 0. 2. 2. 2. 2. 0. 0. 1. 1. 1. 1.]
 [0. 1. 0. 0. 2. 2. 2. 2. 0. 0. 1. 1. 1. 1.]
 [0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 0.]
 [0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 0.]
 [0. 0. 1. 1. 2. 2. 1. 1. 1. 1. 2. 2. 1. 1.]
 [0. 0. 1. 1. 2. 2. 1. 1. 1. 1. 2. 2. 1. 1.]
 [0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1.]
 [0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1.]]


**Question:**

**For the mesh shown in the figure propose different node numbering schemes and identify the resulting changes in the size of the half-band in the stiffness matrix. Assume that each element subroutine is full of $1$s.**

<center><img src="img/halfband.png" alt="files" style="width:300px"></center>


### Glossary of terms.

**Boundary conditions array IBC():** Integer type array storing equation numbers assigned to each nodal point in the mesh.

**Connectivity array IELCON():** Integer type array storing identifiers for the nodal points defining each element in the mesh.

**Assembly:** Computational procedure by which the elemental stiffness matrix are properly added together to form the global stiffness matrix.

**Assembly operator DME():** Integer type array storing the nodal connectivities from each element but translated into equation numbers through the boudnary conditions array **IBC()**.

## Class activity.

* (i) Use the subroutines developed previously to compute the stiffness matrix of bi-linear and cuadratic finite elements to compute the global stiffness matrix for the sample problem discussed in this notebook and with the input files **Snodes.txt and Selements.txt** provided.

* (ii) Assume nodal values for the active displacemnts and use the global matrix found in step (i) to find the internal forces vector $\left\{F_\sigma\right\}$ consistent with the element stresses.

* (iii) Repeat step (ii) but instead of assuming known nodal displacements find them after applying point forces along degrees of freedom $9$, $11$ and $13$ and solving the system of equations:


$$
\left[K^G\right]\left\{U^G\right\} = \left\{F\right\}.
$$


* (iv) Verify that the nodal displacemnts $U^G$ found in step (iii) produce internal forces $\left\{F_\sigma\right\}$ in equilibrium with the external forces $\left\{F\right\}.$


### References

* Bathe, Klaus-Jürgen. (2006) Finite element procedures. Klaus-Jurgen Bathe. Prentice Hall International.

* Juan Gómez, Nicolás Guarín-Zapata (2018). SolidsPy: 2D-Finite Element Analysis with Python, <https://github.com/AppliedMechanics-EAFIT/SolidsPy>.

In [9]:
from IPython.core.display import HTML
def css_styling():
    styles = open('./nb_style.css', 'r').read()
    return HTML(styles)
css_styling()